# Webscraping Toronto Boroughs and Neighborhoods from Wikipedia

The following exercise is to webscrape the data from wikipedia and load it into a Dataframe with required transformations.

#### Importing libraries 

In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [6]:
r = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(r.text, 'html.parser')

#### Webscraping to Dataframe

In [72]:
toronto_lst = []
for row in soup.find('table').find_all('tr'):
    line = row.find_all('td')
    if len(line)!=0:
        toronto_lst.append([line[0].text.replace("\n", ""), line[1].text.replace("\n", ""), line[2].text.replace("\n", "")])
df = pd.DataFrame.from_records(toronto_lst).rename(columns={0:'PostalCode',1:'Borough',2:'Neighborhood'})
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Removing Boroughs with not assigned

In [71]:
df = df[df['Borough']!='Not assigned'].reset_index(drop=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### Merging the boroughs with multiple postal code

In [70]:
df = df.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### Assign neighborhood name as the borough is neighborhood is not assigned

In [69]:
for _,row in df.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### Display the shape of the dataframe

In [73]:
df.shape

(180, 3)